# Create Connection To Postgres

In [1]:
import pandas
from connect_sql_db import build_engine
import time

In [2]:
engine = build_engine(database_name="database1",host="35.225.193.21")

# Read in Yelp Data

In [3]:
buis_df = pandas.read_json("../data/yelp_dataset/yelp_academic_dataset_business.json", lines = True)

In [5]:
review_df = pandas.read_json("../data/yelp_dataset/yelp_academic_dataset_review.json", lines = True)

# Python SQL Functions

In [7]:
#return yelp dataframes dictionary from sql
def read_in_tables(table_dict):
    return_me = {}
    for name,dataframe in table_dict.items():
        return_me[name] = pandas.read_sql(f'select * from "{dataframe}"',con=engine)
        
    return return_me

#send dataframe dictionary to sql
def send_in_dataframes(tables_dict):
    for name,dataframe in tables_dict.items():
        dataframe.to_sql(f"{name}", con = engine, index = False, method='multi',chunksize=1000)
        print(f"{name} df sent...")
    return "dataframe data sent"

#drop dataframe dictionary from sql
def drop_dataframes(tables_dict):
    with engine.connect() as connection:
        for name in tables_dict.keys():
            connection.execute(f'DROP TABLE "{name}"')
            print(f"{i} dropped...")
    return "tables dropped"

# ETL PROCESS

In [ ]:
#transform dictionaries into string
for name,dataframe in yelp_dfs.items():
    for column in dataframe.dtypes[dataframe.dtypes == "object"].index.tolist():
        dataframe[column] = dataframe[column].astype(str)
    

In [ ]:
#create dictionary of cleaned dataframes
yelp_dfs = {
    "business_table": buis_df,
    "checkin_table": checkin_df,
    "review_table": review_df,
    "user_table": user_df,
    "tip_table": tip_df
}

In [ ]:
send_in_dataframes(yelp_dfs)

In [8]:
#testing

In [13]:
buis_df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [46]:
review_df.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [53]:
attributes = buis_df.attributes.tolist()

In [57]:
categories = buis_df.categories.tolist()

In [14]:
buis_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [10]:
review_df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,xQY8N_XvtGbearJ5X4QryQ,OwjRMXRC0KyPrIlcjaXeFQ,-MhfebM0QIsKt87iDN-FNw,2,5,0,0,"As someone who has worked with many museums, I...",2015-04-15 05:21:16
1,UmFMZ8PyXZTY2QcwzsfQYA,nIJD_7ZXHq-FX8byPMOkMQ,lbrU8StCq3yDfr-QMnGrmQ,1,1,1,0,I am actually horrified this place is still in...,2013-12-07 03:16:52
2,LG2ZaYiOgpr2DK_90pYjNw,V34qejxNsCbcgD8C0HVk-Q,HQl28KMwrEKHqhFrrDqVNQ,5,1,0,0,I love Deagan's. I do. I really do. The atmosp...,2015-12-05 03:18:11
3,i6g_oA9Yf9Y31qt0wibXpw,ofKDkJKXSKZXu5xJNGiiBQ,5JxlZaqCnk1MnbgRirs40Q,1,0,0,0,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g...",2011-05-27 05:30:52
4,6TdNDKywdbjoTkizeMce8A,UgMW8bLE0QMJDCkQ1Ax5Mg,IS4cv902ykd8wj1TR0N3-A,4,0,0,0,"Oh happy day, finally have a Canes near my cas...",2017-01-14 21:56:57


In [68]:
categories_lst = []
for i in categories:
    if i != None:
        for v in i.split(","):
            categories_lst.append(v)


In [69]:
set(categories_lst)

{'Caricatures',
 ' Nightlife',
 ' Rolfing',
 ' Lactation Services',
 'Playgrounds',
 ' Pretzels',
 ' Soccer',
 'Pan Asian',
 'Emergency Rooms',
 'Henna Artists',
 'Massage',
 'Teeth Whitening',
 ' Battery Stores',
 'Employment Agencies',
 'Dive Bars',
 ' Basketball Courts',
 'Karate',
 ' Dentists',
 ' Mortgage Brokers',
 'Counseling & Mental Health',
 'Food Banks',
 ' Chinese Martial Arts',
 ' Dive Bars',
 ' Spray Tanning',
 ' Pop-up Shops',
 'Television Stations',
 'Test Preparation',
 'Float Spa',
 'Town Car Service',
 'Transportation',
 'Auto Upholstery',
 ' Knife Sharpening',
 ' Outdoor Gear',
 ' House Sitters',
 'Tennis',
 ' Chiropractors',
 ' Irish Pub',
 ' Packing Supplies',
 ' Skating Rinks',
 'Check Cashing/Pay-day Loans',
 'Rheumatologists',
 ' Laundromat',
 ' Sewing & Alterations',
 ' Flight Instruction',
 ' Phlebologists',
 'Endodontists',
 'Cryotherapy',
 ' Nudist',
 ' Movers',
 'Cocktail Bars',
 ' Private Jet Charter',
 ' DIY Auto Shop',
 ' Wigs',
 'Home & Rental Insuranc

In [87]:
for i in buis_df.categories.tolist():
    if i != None:
        if "restaurants" in i.lower().strip():
            print(i)

Food, Ethnic Food, Specialty Food, Restaurants, Thai
Thai, Restaurants
Restaurants, Thai
Restaurants, Thai
Thai, Restaurants
Sushi Bars, Chinese, Bars, Thai, Karaoke, Restaurants, Vietnamese, Nightlife
Thai, Restaurants
Restaurants, Thai
Thai, Restaurants
Restaurants, Thai, Seafood, Chinese
Thai, Restaurants, Sushi Bars, Japanese
Fast Food, Thai, Restaurants
Asian Fusion, Thai, Japanese, Sushi Bars, Restaurants, Steakhouses
Imported Food, Food, Restaurants, Ethnic Food, Specialty Food, Thai
Thai, Restaurants
Thai, Restaurants
Food, Restaurants, Desserts, Thai
Food, Ethnic Food, Specialty Food, Thai, Restaurants, Food Court
Thai, Vietnamese, Restaurants
Sushi Bars, Asian Fusion, Thai, Restaurants
Thai, Restaurants
Restaurants, Thai, Vegetarian, Vegan
Thai, Restaurants, Vegetarian, Asian Fusion
Restaurants, Thai
Restaurants, Thai
Restaurants, Thai
Thai, Sandwiches, Restaurants, Chinese, Noodles
Thai, Restaurants
Restaurants, Japanese, Chinese, Mexican, Thai, Breakfast & Brunch, Seafood, 

Food, Specialty Food, Thai, Restaurants
Imported Food, Restaurants, Ethnic Food, Thai, Specialty Food, Food
Thai, Restaurants
Restaurants, Thai
Chicken Shop, Restaurants, Noodles, Food, Thai
Thai, Restaurants
Restaurants, Thai
Asian Fusion, Restaurants, Thai
Asian Fusion, Sushi Bars, Restaurants, Thai
Restaurants, Thai
Thai, Restaurants, Vietnamese
Thai, Chinese, Restaurants, Japanese
Thai, Restaurants
Thai, Restaurants
Restaurants, Thai
Restaurants, Vietnamese, Thai
Thai, Chinese, Restaurants, Vietnamese, Pan Asian
Restaurants, Thai
Restaurants, Soup, Thai, Salad
Restaurants, Chinese, Indian, Thai
Pubs, Nightlife, Lounges, Bars, Dive Bars, Restaurants, Thai
Thai, Korean, Sushi Bars, Asian Fusion, Ethnic Food, Restaurants, Specialty Food, Food
Asian Fusion, Malaysian, Restaurants, Thai
Restaurants, Thai
Thai, Restaurants
Restaurants, Thai, Food
Thai, Restaurants
Restaurants, Thai, Sushi Bars
American (New), Bars, Nightlife, Canadian (New), Thai, Restaurants, Dive Bars
Noodles, Thai, Re

In [92]:
def myfunct(x):
    if x != None:
         if "restaurants" in x.lower().strip():
                return True
    else:
        return False

    
buis_df["res"] = buis_df.categories.apply(lambda x: myfunct(x))

In [99]:
new_df = buis_df.loc[buis_df.res == True]

In [106]:
new_df.drop(["attributes","categories","hours"], axis=1, inplace = True)

/usr/local/Caskroom/miniconda/base/envs/minicondavenv_bootcamp/lib/python3.7/site-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
new_df.to_sql(
    "yelp_restaurants", 
    con = engine,
    if_exists = "append",
    chunksize=1000, 
    method='multi',
    index = False
)